In [1]:
#pip install biopython
from Bio import SeqIO
from Bio.Seq import Seq
# https://biopython.org/wiki/SeqIO
import pandas as pd
import numpy as np
import re

In [2]:
# OPEN TATE PRIMERS 
Primers = pd.read_excel ("/Users/qchen2/Desktop/DesignPrimers/TatePrimer.xlsx")
#print (Primers.head())
# I think the information need here are 
# Target G27 Gene
Target_Gene = Primers["Target G27 Gene"]
Target_Gene = Target_Gene.tolist()
# print (len(Target_Gene))
# Expect 1152, pop out 1152
# Primer 3 
Primer3 = Primers["(3) Reverse inside"]
# print (len(Primer3))
# Correct size
# Primer 4
Primer4 = Primers["(4) Forward outside"]
#print (Primer4.head())
#print (Primer3.head())
#print (Target_Gene.head())

In [3]:
# OPEN G27 GENOME
file_path = "/Users/qchen2/Desktop/DesignPrimers/G27.gb"
# I want to see the structure of this file. 
with open(file_path, "r") as file:
    # the file here is known as "handle" in some tutorial
    # Genbank because that is the type of file I am importing. 
    for record in SeqIO.parse(file, "genbank"):
        G27 = record.seq 
        # print record.seq
# The record.seq is calling out the information with the category (?) called seq
# You can also do record.id or record.name and etc. 
# Make the sequence of G27 be G27
#G27 = record.seq
# Testing to see if it opened the right thing
# Proof of concept, check the map to see the sequence actually are the Nth in the file.
#print(G27[55:68])

In [4]:
# Import Primer Library summary 
Summary = pd.read_excel ("/Users/qchen2/Desktop/DesignPrimers/Library_Sum.xlsx")
HPG27_Gene = Summary["HPG27 Gene"]

In [13]:
SEQ = []
SEQ.append(str(G27))
SEQ = ''.join(SEQ)
PM = Primer3.tolist()
# If there is a problem, test with 5 rows first by activating the following code
############################# TEST WITH 5
#PM = PM[0:5]
PM2 = Primer4.tolist()
############################# TEST WITH 5
#PM2 = PM2[0:5]
antiSEQ = []
antiSEQ.append(str(G27.reverse_complement()))
antiSEQ = ''.join(antiSEQ)

Loc  = {}
for index, primer in enumerate(PM):
    matches = list(re.finditer(str(primer), SEQ, flags=re.IGNORECASE))
    if matches:
        for match in matches:
            Loc[index] = match.start()+1, match.end()
    else:
        # If the primers are not written in 5'>3" end, use (primer[::-1], antiSEQ) here
        matches_anti = list(re.finditer(str(primer), antiSEQ, flags=re.IGNORECASE)) 
        if matches_anti:
            for match in matches_anti:
                Loc[index] = abs(match.start() - len(SEQ)), abs(match.end() - 1 - len(SEQ))          
        else:
            #print(index, primer, "Warning, No match")
            Loc[index] = "Warning, No match"
            
for index, primer in enumerate(PM2):
    matches = list(re.finditer(str(primer), SEQ, flags=re.IGNORECASE))
    if matches:
        for match in matches:
            new_value = list(Loc[index])
            new_value.append(match.start()+1)
            new_value.append(match.end())
            Loc[index] = sorted(new_value)
            
    else:
        # If the primers are not written in 5'>3" end, use (primer[::-1], antiSEQ) here
        matches_anti = list(re.finditer(str(primer), antiSEQ, flags=re.IGNORECASE)) 
        if matches_anti:
            for match in matches_anti:
                new_value = list(Loc[index])
                new_value.append(abs(match.start() - len(SEQ)))
                new_value.append(abs(match.end() -1 - len(SEQ)))
                Loc[index] = sorted(new_value)
        else:
            #print(index, primer, "Warning, No match")
            Loc[index] = "Warning, No match"
# If the code does not run, it could be you should activate the warning list
# This will help identify the primer set that has no match
# and you can go manually to that set to see why there is no match

#warning_list = []
middle_values_dict = {}

for key, value in Loc.items():
    middle_values_dict[key] = (value[1], value[2]) 
    
#for key, value in Loc.items():
#    if isinstance(value, (list, tuple)) and len(value) == 4:
#        middle_values_dict[key] = (value[1], value[2]) 
#    else:
#        warning_list.append(key) 

# Here are more trouble shooting codes in case it does not run 
#print("Primer Locations:", Loc)
#print("Marker location:", middle_values_dict)
#print ("Primer 3 is", PM)
#print ("Primer 4 is", PM2)
#print (warning_list)
#print (middle_values_dict)

In [14]:
print (len(Loc.values()))
print (len(Target_Gene))
print (len(middle_values_dict.values()))

1152
1152
1152


In [15]:
# Using zip to match locations to target gene (on the primer list)
new_dict = dict(zip(Target_Gene, middle_values_dict.values()))
#print (new_dict)

In [16]:
# Merge locations to HPG27 Gene (on the summary list) 
value_df = pd.DataFrame.from_dict(new_dict, orient='index', columns=['LOC5', 'LOC3'])
value_df.reset_index(inplace=True)
value_df.rename(columns={'index': 'HPG27 Gene'}, inplace=True)
merged_df = Summary.merge(value_df, how='left', left_on="HPG27 Gene", right_on="HPG27 Gene", )
merged_df.to_excel("SUM.xlsx", index=False)